# 15 Foundational Leetcode Patterns
Learning when to apply these patterns is foundational to good software engineering.

In [80]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd()))
from src.utils import timer

____
## 0. Building Strings
- Since strings are immutable, concatenating a single character to a string of length $n$ is an $O(n)$ operation.
- Many problems involve building a string, piece by piece.

*What is the best way to build strings incrementally?*

This varies between languages, so we will focus on Python.

#### How to Build a String in Python
1. Declare a list
2. When building the string, add characters to the list.
   1. Adding characters to a list is $O(1)$, so across $n$ operations, we have $O(n)$
3. Once we have everything, we convert to a string with `"".join(list)`. This is $O(n)$.
4. In total, we get $O(n + n) = O(2n) = O(n)$.

____
## 1. Prefix Sum
#### Used for: ***querying the sum of elements in a subarray.***

- The concept of a *prefix sum* is equivalent to the concept of a *sequence of partial sums* from the study of series in math.
- Partial sums are useful in determining infinite series' convergence properties.
- Prefix sums are useful to use as *lookup tables* when querying the sum of elements in a subarray.
  - Prefix sums allow us to find the sum of any subarray in $O(1)$.
  - This is because **`sum[i,j]` (inclusive) can be expressed as a difference of partial sums**.
- As a general note, when a subarray starts at index 0, it is considered a *prefix* of the array.

Say we are given `A`, and we need to find the sum of elements `A[i : j]`, for several given pairs of $(i, j)$.
```python
A = [1, 2, 3,  4,  5,  6,  7]
P = [1, 3, 6, 10, 15, 21, 28]
```

### Naive Solution

In [81]:
A = list(range(1, 8))
QUERIES = [(2, 4), (2, 5), (2, 6)]

@timer
def find_subarray_sum(arr: list, i: int, j: int) -> int:
    """Equivalent to `sum(A[i:j+1])`."""
    tot = 0
    for num in arr[i : j + 1]:
        tot += num
    return tot

for i, j in QUERIES:
    find_subarray_sum(A, i, j)

                       find_subarray_sum: 12 found in 2.09989957511425e-06 seconds
                       find_subarray_sum: 18 found in 1.800013706088066e-06 seconds
                       find_subarray_sum: 25 found in 3.200024366378784e-06 seconds


*What is the time complexity of the naive solution if there is only 1 query?*
- $O(n)$ because in the worst case, it must loop through the entire length of `A`.

*What if we have multiple queries?*
- Then the time complexity grows to $O(mn)$, since $m$ queries must loop through $n$ elements.

*How can we answer these queries faster?*
- By applying the prefix sum pattern.

*What makes the prefix sum pattern fast?*
- It takes advantage of a property about sums and their relations to prefix sums.
- This property (i.e. formula) allows for all subarray sum queries to be computed in $O(1)$.

*What is the special property about sums and how they relate to prefix sums?*
```python
Notice
           P[i] = A[0] + A[1] + ... + A[i]
           P[j] = A[0] + A[1] + ... + A[j]
    P[j] - P[i] = A[0] + A[1] + ... + A[j] - (A[0] + A[1] + ... + A[i])
                = A[i + 1] + A[i + 2]+ ... + A[j]
P[j] - P[i - 1] = A[i] + A[i + 1] + ... + A[j]
                = sum(A[i : j]) (inclusive)

Now notice
       P[i - 1] = A[0] + A[1] + ... + A[i - 1]
                = P[i] - A[i]

So
 sum(A[i : j])  = P[j] - P[i] + A[i]
```

*Why is `P[j] - P[i] + A[i]` better than `P[j] - P[i - 1]`?*
- With `P[j] - P[i] + A[i]`, we don't need to deal with the pesky case when `i=0`, which would cause an error with `P[i - 1]`.

*What is the best way to remember how this property is derived?*
- The best general path to follow is:
  - Find the difference `P[j] - P[i]`.
  - Notice the difference is the `sum[i + 1 : j]` (inclusive).
  - Notice `P[i - 1]` can be rewritten as `P[i] - A[i]`.
  - Substitute this new definition to get `P[j] - P[i] + A[i]`.
- This means that ***to answer a subsequence sum query, we can sum just 3 elements*** rather than $n$ consecutive elements.

*What is the time complexity of our query problem, when using prefix sums?*
- Building the prefix sum array costs $O(n)$ to build, but then answering each query is $O(1)$.
- This changes the time complexity to $O(n + m)$.

### Optimized Solution

In [82]:
from typing import List, Tuple

@timer
def create_prefix_sum(arr):
    for i in range(1, len(arr)):
        arr[i] += arr[i - 1]
    return arr

@timer
def find_subarray_sums (arr: list, queries: List[Tuple[int, int]]):
    arr_ps = create_prefix_sum(arr)  # Loops through n elements once.
    results = []
    for i, j in queries: # Loops through m elements once.
        if i == 0:  # Return the partial sum directly if starting from the beginning.
            results.append(arr_ps[j])
        results.append(arr_ps[j] - arr_ps[i - 1])
    return results

A = list(range(1, 8))
QUERIES = [(2, 4), (2, 5), (2, 6)]
find_subarray_sums(A, QUERIES)


                       create_prefix_sum: [1, 3, 6, 10, 15, 21, 28] found in 4.800036549568176e-06 seconds
                      find_subarray_sums: [12, 18, 25] found in 0.0003162000793963671 seconds


[12, 18, 25]

*What is the main lesson?*
- We don't always need a new array for a new query.
- We can *pre-process* data initially to save resources in the long run.
- Note that building a prefix sum is a form of *pre-processing*.
- A prefix sum array is meant to be used as a *lookup table* to save resources.

### Recommended Leetcode Practice Problems
 - 303. ~~**Range Sum Query - Immutable** (`Easy`)~~
 - 525. ~~**Contiguous Array** (`Medium`)~~
 - 560. **Subarray Sum Equals K** (`Medium`)

____
## 2. Two Pointers

This pattern is used for: ***efficiently traversing or comparing elements within a sequence***.

The pattern is to: ***start the pointers at the edges of the input, then move them towards each other until they meet***.

As instructions:
1. Start one pointer at the first index `0` and the other pointer at the last index `input.length - 1`.
2. Use a *while loop* until the *pointers are equal* to each other.
3. At each iteration of the loop, *move at least one of the pointers* ***toward*** each other (or both), depending on given criteria.

Say we are given `A`, a string of characters, and we want to check whether it is a palindrome.
A string is a palindrome if it's the same when the order of characters is reversed.
```python
A = "a b c d c b a"
```


### Optimized Solution
To solve this, start the pointers at the edges of the input. Move them towards each other until they meet. If everything has been equal up until this final character, then the string is a palindrome.

In [83]:
def is_palindrome(mystr):
    n = len(mystr)
    start = 0
    end = n - 1
    while start < end:  # a b c b a
        if mystr[start] != mystr[end]:
            return False
        start += 1
        end -= 1
    return True

*How does using the two pointers approach affect time complexity?*
- We often see that $O(n^2)$ reduces to $O(n)$.
- Since the pointers start $n$ away from each other and move at least one step closer every interation, this technique will never have more than $O(n)$ iterations.
  - This means if we can limit each iteration to $O(1)$ (direct address lookup), we will get a linear runtime.

*What about space complexity?*
- $O(1)$, because no matter how big the input is, we only use *two* integer variables.

### Recommended Leetcode Practice Problems
 - 167. **Two Sum II - Input Array Is Sorted** (Medium)
 - 15. **3Sum** (Medium)
 - 11. **Container With Most Water** (Medium)

____
2.1 Two Pointers, Two Arrays

This pattern is to: ***check along two arrays simultaneously until all elments have been checked***.

As Instructions:
1. Create two pointers, one for each iterable. Each pointer should start at the first index.
2. Use a while loop until one of the pointers reaches the end of its iterable.
3. At each iteration of the loop, move the pointers forward. This means incrementing either one of the pointers or both of the pointers. Deciding which pointers to move will depend on the problem we are trying to solve.
4. Because our while loop will stop when one of the pointers reaches the end, the other pointer will not be at the end of its respective iterable when the loop finishes. Sometimes, we need to iterate through all elements - if this is the case, you will need to write extra code here to make sure both iterables are exhausted.

As Pseudocode:
```
function fn(arr1, arr2):
    i = j = 0
    while i < arr1.length AND j < arr2.length:
        Do some logic here depending on the problem
        Do some more logic here to decide on one of the following:
            1. i++
            2. j++
            3. Both i++ and j++

    // Step 4: make sure both iterables are exhausted
    // Note that only one of these loops would run
    while i < arr1.length:
        Do some logic here depending on the problem
        i++

    while j < arr2.length:
        Do some logic here depending on the problem
        j++
```

____
## 3. Sliding Window
This pattern is used for: ***finding subarrays or substrings that meet certain requirements.***.

### Intuition
- The idea behind sliding window is to consider *only* valid subarrays.
- A subarray can be defined by two pointers: `left` and `right`
- These points represent the current subarray under consideration.
- Initially, `left=right=0`.
- When we expand the size of the window, we do it by incrementing `right`.
- If the window then becomes invalid, we can remove elements by incrementing `left`.
- As we add and remove elements, the window is sliding from left to right.
- *However*, the window's size is constantly changing!
    - It grows as large as it can until it's invalid, then it shrinks.
    - This might be better thought of as "Inchworm".

### Pseudocode
```
function fn(arr):
    left = 0
    for (int right = 0; right < arr.length; right++):
        Do some logic to "add" element at arr[right] to window

        while WINDOW_IS_INVALID:
            Do some logic to "remove" element at arr[left] from window
            left++

        Do some logic to update the answer
```

Say we are given an array of integers `A`, and we need to find the subarray of size `k` with the maximum sum. For simplicity, we assume that there is only one such maximum.
```python
A = [3, 2, 7, 5, 9, 6, 2], k=3
```


### Naive Solution
The naive method would be to consider all the subarrays of size 3, using a nested for loop.

In [84]:
@timer
def max_subarr_sum(arr, k):
    n = len(arr)
    max_sum = float("-inf")
    for i in range(n - k + 1):
        current_sum = 0
        for j in range(k):
            current_sum += arr[i + j]
        if current_sum > max_sum:
            max_sum = current_sum
            max_start_index = i
    return arr[max_start_index : max_start_index + k], max_sum

A = [3, 2, 7, 5, 9, 6, 2]
k = 3
max_subarr_sum(A, k)

                          max_subarr_sum: ([7, 5, 9], 21) found in 1.71000137925148e-05 seconds


([7, 5, 9], 21)

*What is the time complexity of the naive solution?*
- $O(nk)$, since for each of the n start indices, there are k traversals.

### Optimized Solution
The sliding window approach keeps track of a `window_sum` that only slides once across the entire array. We can visualize as follows:
```python
       0   1   2   3   4   5   6
A = [  3,  2,  7,  5,  9,  6,  2 ], k=3
       |-------|   .   .   .   .
           |-------|   .   .   .
               |-------|   .   .
                   |-------|   .
                       |-------|
```

In [85]:
@timer
def max_subarr_sum_sliding_window(arr, k):
    n = len(arr)
    window_sum = sum(arr[:k])
    max_sum = window_sum
    max_start_index = 0
    for i in range(n - k):
        window_sum = window_sum - arr[i] + arr[i + k]
        if window_sum > max_sum:
            max_sum = window_sum
            max_start_index = i
    return arr[max_start_index : max_start_index + k], max_sum

A = [3, 2, 7, 5, 9, 6, 2]
k = 3
max_subarr_sum_sliding_window(A, k)

           max_subarr_sum_sliding_window: ([2, 7, 5], 21) found in 7.800059393048286e-06 seconds


([2, 7, 5], 21)

*How does this reduce the time complexity?*
- Notice that for any array, there are $\frac{n}{2}(n+1)$ subarrays.
  - ($n$ length 1 arrays) + ($n - 1$ length 2 arrays) + ... + (1 length $n$ array).
- Therefore, any algorithm that looks at *every* subarray will be at least $O(n^2)$.
- A sliding window *guarantees* a max of $2n$, since the right pointer can move at maximum $n$ times, and same for the left.
- If the logic for each window is $O(1)$, then sliding window runs in $O(n)$.
  - $O(nk) \rightarrow O(n)$

*What is the space complexity?*
- Since we only use 3 variables, the space complexity is constant, $O(1)$.



### Number of Subarrays
If a problem asks for ***the number of subarrays***, we can use a sliding window, but we need a math trick.

- Let's say we currently have a window `(lp, rp)`. *How many windows **end** at`rp`?*
- There's the current window `(lp, rp)`, then `(lp + 1, rp)`, `(lp + 2, rp)`, ..., `(rp, rp)`.
- This means we can *fix* `rp` and then choose *any* value in [`lp`,`rp`].
- Thus, the number of valid windows ***ending*** at `rp` is equal to the size of the window: `rp - lp + 1`.

### Fixed Window Size
- Rather than an inchworm, this is more of a traditional, solid window pane. These problems are usually easier, and $O(n)$ is optimal.

#### Example: Length K Subarray With Max Sum
- Given an integer array `nums` and an integer `k`, find the sum of the subarray with the largest sum whose length is `k`.

```python
def find_max_subarr_sum(nums, k) -> int:
    n = len(nums)
    ans = curr = sum(nums[:k])
    for i in range(k, n):
        curr += nums[i] - nums[i - k]  # Start at the END of the first window!
        ans = max(ans, curr)  # This has a more natural feeling of ending at `n`.
    return ans
```

### Recommended Leetcode Practice Problems
 - 643. **Maximum Average Subarray I** (`Easy`)
 - 3. **Longest Substring Without Repeating Characters** (`Medium`)
 - 76. **Minimum Window Substring** (`Hard`)

____
## 4. Fast and Slow Pointers
This pattern is used for: ***finding cycles within linked lists or arrays***.
- This pattern is similar to the two pointers pattern, but the pointers move at different speeds.
- As an analogy, imagine two runners going around a track, but one runs twice as fast as the other.
  - Eventually, one of the runners will catch up with the slower one and pass them, over and over.
- This approach can check if a linked list contains a cycle.
  - The slow pointer moves one node at a time, when the fast pointer moves two nodes at a time.
  - If there is a cycle, the two nodes will eventually meet.
- This method can also find the middle pointer of a linked list in one pass.
  - When the fast pointer reaches the end, the slow pointer will be in the middle.

### Recommended Leetcode Problems
 - 141. **Linked List Cycle** (`Easy`)
 - 202. **Happy Number** (`Easy`)
 - 287. **Find the Duplicate Number** (`Medium`)

____
## 5. Linked List In-place Reversal

### Recommended Leetcode Problems
 - 141. **Reverse Linked List** (`Easy`)
 - 202. **Reverse Linked List II** (`Easy`)
 - 287. **Swap Nodes in Pairs** (`Medium`)

____
## 6. Monotonic Stack

### Recommended Leetcode Problems
 - 141. **Next Greater Element I** (`Easy`)
 - 202. **Daily Temperatures** (`Medium`)
 - 287. **Largest Rectangle in Histogram** (`Hard`)

____
## 7. Top K Elements

### Recommended Leetcode Problems
 - 215. **Kth Largest Element in an Array** (`Medium`)
 - 347. **Top K Frequent Elements** (`Medium`)
 - 373. **Find K Pairs with Smallest Sums** (`Medium`)

____
## 9. Modified Binary Search

### Recommended Leetcode Problems
 - 33. **Search in Rotated Sorted Array** (`Medium`)
 - 153. **Find Minimum in Rotated Sorted Array** (`Medium`)
 - 240. **Search a 2D Matrix II** (`Medium`)

____
## 8. Overlapping Intervals

### Recommended Leetcode Problems
 - 56. **Merge Intervals** (`Medium`)
 - 57. **Insert Interval** (`Medium`)
 - 435. **Non-overlapping Intervals** (`Medium`)

____
## 10. Binary Tree Traversal

### Recommended Leetcode Problems
 - 257. **Binary Tree Paths** (`Easy`) [PreOrder]
 - 230. **Kth Smallest Element in a BST** (`Medium`) [InOrder]
 - 124. **Binary Tree Maximum Path Sum** (`Hard`) [PostOrder]
 - 107. **Binary Tree Level Order Traversal II** (`Medium`) [LevelOrder]

____
## 11. Depth-First Search (DFS)

### Recommended Leetcode Problems
 - 133. **Clone Graph** (`Medium`)
 - 113. **Path Sum II** (`Medium`)
 - 210. **Course Schedule II** (`Medium`)

____
## 12. Breadth-First Search (BFS)

### Recommended Leetcode Problems
 - 133. **Binary Tree Level Order Traversal** (`Medium`)
 - 113. **Rotting Oranges** (`Medium`)
 - 210. **Word Ladder** (`Hard`)

____
## 13. Matrix Traversal

### Recommended Leetcode Problems
 - 733. **Flood Fill** (`Easy`)
 - 200. **Number of Islands** (`Medium`)
 - 130. **Surrounded Regions** (`Medium`)

____
## 14. Backtracking

This pattern is used for: ***exploring all potential solution paths and backtracking the paths that do not lead to a valid solution***.

### Recommended Leetcode Problems
 - 46. **Permutations** (`Medium`)
 - 78. **Subsets** (`Medium`)
 - 51. **N-Queens** (`Hard`)

____
## 15. Dynamic Programming (DP)

This pattern is used for: ***solving optimization problems by breaking them down into smaller sub-problems and storing their solutions to avoid repetitive work***.

DP is a huge subject, so there are some common patterns under this larger umbrella we should know about:
- Fibonacci Numbers
- 0/1 Knapsack
- Longest Common Subsequence (LCS)
- Longest Increasing Subsequence (LIS)
- Subset Sum
- Matrix Chain Multiplication

### Recommended Leetcode Problems
 - 70. **Climbing Stairs** (`Medium`)
 - 300. **Longest Increasing Subsequence** (`Medium`)
 - 322. **Coin Change** (`Hard`)
 - 416. **Partition Equal Subset Sum** (`Medium`)
 - 1143. **Longest Common Subsequence** (`Medium`)
 - 312. **Busrt Balloons** (`Hard`)

____
## 16. Hashing
This pattern is used for: ***tracking things we've already encountered***.
- A subpattern of this is ***tracking the frequency of things (i.e. counting)***.
  - In Python, using a `collections.Counter` (a subclass of `dict`), is the best for types of problems involve finding the element counts in an array, or the character counts in a string.

- A hash map is an unordered data structure that stores key-value pairs.
- With arrays, we map indices to values, and with hash maps, we map keys to values.
  - A key can be almost anything (and it should be immutable).
    - Some languages allow for mutable keys, but keeping them immutable is a good rule of thumb.
  - Values can be *anything*.

*What are the disadvantages of using hash maps?*
 - For smaller input sizes, they can be slower due to overhead.
 - There are things called *collisions* too.
 - Hash tables can take up more space than dynamic arrays, since a hash table's size limit is set by the programmer.
 - Re-sizing a hash table is expensive since every existing key needs to be re-hashed.
 - When you don't know how many elements you need to store, arrays are more flexible with resizing and conserving space.

*What are collisions?*
- When different keys convert to the same integer, it is called a collision.
- If these are not handled, older keys will be overridden and data will be lost.
- There are multiplle ways to handle collisions, but the most common is *chaining*.
- Collisions *must* be handled, but doing so is costly.

*How can we minimize collisions?*
- Ensure ***the size of the hash table's array and modulus is a prime number***.
  - For example: 10**4 + 7, 10**6 + 3, 10**9 + 7

*What is chaining?*
- We store linked lists inside the hash map's array instead of the elements themselves.
- If there are collisions, the collided key-value pairs are linked together in a linked list.

*Are sets hash tables?*
- No, but they use the same mechanism for hashing keys into integers.
- Unlike a hash table, sets *don't map their keys to anything*.
- Sets are more convenient when all we care about it checking for existence.

*What if we really need to use arrays as key?*
- In Python, it's easiest to just convert the array (list) into its immutable analogue, the tuple.